In [1]:
from confluent_kafka import Consumer, KafkaError
import psycopg2
import json
import random
import time

# Configuration du consommateur Kafka
conf = {
    'bootstrap.servers': 'kafka:9092',
    'group.id': 'my_group',  # Assurez-vous que ce groupe est unique pour vos tests
    'auto.offset.reset': 'earliest',
}

# Création d'une instance de consommateur Kafka
print("Création du consommateur Kafka...")
consumer = Consumer(conf)

# Inscription au topic
topic = 'Mostodon_topic_stream'
print(f"Inscription au topic: {topic}...")
consumer.subscribe([topic])

# Fonction pour établir une connexion à PostgreSQL
def connect_to_postgres():
    print("Connexion à PostgreSQL...")
    while True:
        try:
            conn = psycopg2.connect(
                host="postgres",
                database="DB_Mastodon",
                user="fadi",
                password="fadi"
            )
            print("Connexion à PostgreSQL réussie.")
            return conn
        except Exception as e:
            print(f"Erreur de connexion à PostgreSQL: {e}. Tentative de reconnexion dans 5 secondes...")
            time.sleep(5)

conn = connect_to_postgres()
cursor = conn.cursor()

# Création de la table si elle n'existe pas déjà (sans created_at)
create_table_query = '''
CREATE TABLE IF NOT EXISTS Mostodon_BRONZ_bis (
    id BIGINT PRIMARY KEY,
    username TEXT NOT NULL,
    display_name TEXT NOT NULL,
    content TEXT NOT NULL,
    favourites_count INT NOT NULL,
    reblogs_count INT NOT NULL,
    replies_count INT NOT NULL
);
'''

try:
    cursor.execute(create_table_query)
    conn.commit()
    print("Table Mostodon_BRONZ_bis créée ou déjà existante.")
except Exception as e:
    print(f"Erreur lors de la création de la table: {e}")
    raise

try:
    while True:
        # Lire les messages du topic
        msg = consumer.poll(1.0)  # Timeout de 1 seconde
        
        if msg is None:
            print("Aucun message reçu, en attente...")  # Log pour l'absence de message
            continue  # Pas de message reçu

        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                print("Atteint la fin de la partition, pas de nouveaux messages.")
                continue
            else:
                print(f"Erreur Kafka: {msg.error()}")
                continue
        
        # Traiter le message
        message_value = msg.value().decode('utf-8')
        try:
            print(f"Message brut reçu : {message_value}")  # Debug : afficher le message brut
            message_dict = json.loads(message_value)
            print(f"Message décodé : {message_dict}")  # Debug : afficher le message décodé
            
            # Afficher les données récupérées (sans la date)
            print("Données récupérées :")
            print(f"ID: {message_dict['id']}")
            print(f"Username: {message_dict['username']}")
            print(f"Display Name: {message_dict['display_name']}")
            print(f"Content: {message_dict['content']}")
            print(f"Favourites Count: {message_dict.get('favourites_count', 'Non spécifié')}")
            print(f"Reblogs Count: {message_dict.get('reblogs_count', 'Non spécifié')}")
            print(f"Replies Count: {message_dict.get('replies_count', 'Non spécifié')}")

            # Vérifier que les champs nécessaires sont présents
            required_fields = ['id', 'username', 'display_name', 'content']
            if not all(field in message_dict for field in required_fields):
                print("Message manquant des champs nécessaires, ignoré.")
                print(f"Champs trouvés : {message_dict.keys()}")  # Debug : afficher les champs trouvés
                continue

            favourites_count = message_dict.get('favourites_count', random.randint(5, 100))
            reblogs_count = message_dict.get('reblogs_count', random.randint(5, 100))
            replies_count = message_dict.get('replies_count', random.randint(5, 100))

            print(f"Insertion du toot : ID={message_dict['id']}, Username={message_dict['username']}, "
                  f"Display Name={message_dict['display_name']}, Content={message_dict['content']}, "
                  f"Favourites Count={favourites_count}, Reblogs Count={reblogs_count}, Replies Count={replies_count}")

            # Requête d'insertion avec gestion du conflit sur l'ID (sans created_at)
            insert_query = '''INSERT INTO Mostodon_BRONZ_bis (id, username, display_name, content, favourites_count, reblogs_count, replies_count) 
                              VALUES (%s, %s, %s, %s, %s, %s, %s) 
                              ON CONFLICT (id) DO NOTHING;'''
            cursor.execute(insert_query, (
                message_dict['id'],
                message_dict['username'],
                message_dict['display_name'],
                message_dict['content'],
                favourites_count,
                reblogs_count,
                replies_count
            ))
            conn.commit()
            print(f"Toot inséré : {message_dict['id']} - {message_dict['username']}")

        except json.JSONDecodeError:
            print("Erreur lors du décodage JSON, message ignoré.")
            continue
        except Exception as e:
            print(f"Erreur lors de l'insertion : {e}")
            conn.rollback()  # Annuler la transaction en cas d'erreur

except KeyboardInterrupt:
    print("Arrêt du consommateur par l'utilisateur.")
except Exception as e:
    print(f"Erreur dans le consommateur: {e}")
finally:
    # Compter le nombre total d'entrées dans la table 'Mostodon_BRONZ_bis'
    print("Calcul du nombre total d'entrées dans la table 'Mostodon_BRONZ_bis'...")
    cursor.execute("SELECT COUNT(*) FROM Mostodon_BRONZ_bis;")
    count = cursor.fetchone()[0]
    print(f"Nombre total d'entrées dans la table 'Mostodon_BRONZ_bis' : {count}")

    # Fermer le consommateur et la connexion à PostgreSQL
    consumer.close()
    cursor.close()
    conn.close()


Création du consommateur Kafka...
Inscription au topic: Mostodon_topic_stream...
Connexion à PostgreSQL...
Connexion à PostgreSQL réussie.
Table Mostodon_BRONZ_bis créée ou déjà existante.
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attente...
Aucun message reçu, en attent

KeyboardInterrupt: 